    Nama   : Ryan Rizky Fathinanto
    Email  : ryanrizkyf@gmail.com
    No. HP : 085692475682

In [1]:
!pip install -q kaggle

In [2]:
import zipfile
import os
import glob 
import warnings

from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import Xception
from tensorflow.keras.layers import AveragePooling2D
from tensorflow.keras.layers import Flatten
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Input
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Activation, Dense, Flatten
from tensorflow.keras.layers import Dropout
from tensorflow.keras.optimizers import Adam
import tensorflow as tf

from keras.preprocessing import image
from google.colab import files
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
%matplotlib inline

In [3]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"ryanrizkyf","key":"a717404121b46b69e6f5b7a8618b8762"}'}

In [4]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json
!ls ~/.kaggle

kaggle.json


In [5]:
!kaggle datasets download -d trolukovich/food11-image-dataset

 98% 1.06G/1.08G [00:04<00:00, 260MB/s]
100% 1.08G/1.08G [00:04<00:00, 250MB/s]


In [6]:
!mkdir foods
!unzip -qq food11-image-dataset.zip -d foods
!ls foods

evaluation  training  validation


In [7]:
!ls foods/training/

 Bread		  Egg	        Noodles-Pasta   Soup
'Dairy product'  'Fried food'   Rice	        Vegetable-Fruit
 Dessert	  Meat	        Seafood


In [8]:
foods_train = os.path.join('/content/foods/evaluation')
foods_validation = os.path.join('/content/foods/validation')

print(os.listdir(foods_train))
print(os.listdir(foods_validation))

['Bread', 'Dessert', 'Meat', 'Dairy product', 'Egg', 'Fried food', 'Vegetable-Fruit', 'Rice', 'Noodles-Pasta', 'Soup', 'Seafood']
['Bread', 'Dessert', 'Meat', 'Dairy product', 'Egg', 'Fried food', 'Vegetable-Fruit', 'Rice', 'Noodles-Pasta', 'Soup', 'Seafood']


In [9]:
train_dir = '/content/foods/training'
test_dir = '/content/foods/evaluation'
validation_dir = '/content/foods/validation'

In [10]:
train_datagen = ImageDataGenerator(
                    rescale=1./255,
                    rotation_range=20,
                    horizontal_flip=True,
                    vertical_flip=True,
                    shear_range = 0.2,
                    zoom_range=0.2,
                    fill_mode = 'nearest')
 
test_datagen = ImageDataGenerator(
                    rescale=1./255)

validation_datagen = ImageDataGenerator(
                    rescale=1./255,
                    rotation_range=20,
                    horizontal_flip=True,
                    shear_range = 0.2,
                    fill_mode = 'nearest')

In [11]:
train_generator = train_datagen.flow_from_directory(
        train_dir,
        target_size=(224, 224),
        batch_size=64,
        class_mode='categorical')
 
validation_generator = test_datagen.flow_from_directory(
        validation_dir,
        target_size=(224, 224),
        batch_size=64,
        class_mode='categorical')

Found 9866 images belonging to 11 classes.
Found 3430 images belonging to 11 classes.


In [12]:
bModel = Xception(include_top=False,
                  weights='imagenet',
                  input_tensor=Input(shape=(224,224,3)))
bModel.summary()

83689472/83683744 [==============================] - 0s 0us/step
Model: "xception"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
block1_conv1 (Conv2D)           (None, 111, 111, 32) 864         input_1[0][0]                    
__________________________________________________________________________________________________
block1_conv1_bn (BatchNormaliza (None, 111, 111, 32) 128         block1_conv1[0][0]               
__________________________________________________________________________________________________
block1_conv1_act (Activation)   (None, 111, 111, 32) 0           block1_conv1_bn[0][0]            
__________________________

In [23]:
hModel = bModel.output
hModel = AveragePooling2D(pool_size=(4, 4))(hModel)
hModel = Flatten()(hModel)
hModel = Dense(128, activation='relu')(hModel)
hModel = Dropout(0.5)(hModel)
hModel = Dense(11, activation='softmax')(hModel)

model = Model(bModel.input, hModel)

for layer in bModel.layers:
  layer.trainable=False

model.summary()

Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
block1_conv1 (Conv2D)           (None, 111, 111, 32) 864         input_1[0][0]                    
__________________________________________________________________________________________________
block1_conv1_bn (BatchNormaliza (None, 111, 111, 32) 128         block1_conv1[0][0]               
__________________________________________________________________________________________________
block1_conv1_act (Activation)   (None, 111, 111, 32) 0           block1_conv1_bn[0][0]            
____________________________________________________________________________________________

In [19]:
#  model = tf.keras.models.Sequential([
#     tf.keras.layers.Conv2D(128, (3,3), activation='relu', input_shape=(150, 150, 3), padding='same'),
#     tf.keras.layers.MaxPooling2D(2, 2),
#     tf.keras.layers.Conv2D(64, (3,3), activation='relu', padding='same'),
#     tf.keras.layers.MaxPooling2D(2,2),
#     tf.keras.layers.Conv2D(64, (3,3), activation='relu', padding='same'),
#     tf.keras.layers.MaxPooling2D(2,2),
#     tf.keras.layers.Conv2D(32, (3,3), activation='relu', padding='same'),
#     tf.keras.layers.MaxPooling2D(2,2),
#     tf.keras.layers.Flatten(),
#     tf.keras.layers.Dense(512, activation='relu'),
#     tf.keras.layers.Dropout(0.5),
#     tf.keras.layers.Dense(11, activation='softmax')
# ])

In [20]:
# model.summary()

In [24]:
class myCallback(tf.keras.callbacks.Callback):
  def on_epoch_end(self, epoch, logs={}):
    if(logs.get('accuracy')>0.92):
      print("\nAccuracy telah mencapai >92%!")
      self.model.stop_training = True
callbacks = myCallback()

In [25]:
optimizer = tf.keras.optimizers.Adam(learning_rate=0.01) 

model.compile(loss='categorical_crossentropy',
              optimizer=optimizer,
              metrics=['accuracy'])

num_epochs=30
hist = model.fit(train_generator,
                 epochs=num_epochs,
                 validation_data=(validation_generator),
                 callbacks=[callbacks],
                 batch_size=128)

Epoch 1/30
155/155 [==============================] - 215s 1s/step - loss: 1.5396 - accuracy: 0.5018 - val_loss: 0.9574 - val_accuracy: 0.7093
Epoch 2/30
155/155 [==============================] - 157s 1s/step - loss: 1.2803 - accuracy: 0.5822 - val_loss: 0.8404 - val_accuracy: 0.7426
Epoch 3/30
155/155 [==============================] - 157s 1s/step - loss: 1.2035 - accuracy: 0.6060 - val_loss: 0.8095 - val_accuracy: 0.7636
Epoch 4/30
155/155 [==============================] - 157s 1s/step - loss: 1.1783 - accuracy: 0.6221 - val_loss: 0.8365 - val_accuracy: 0.7542
Epoch 5/30
155/155 [==============================] - 157s 1s/step - loss: 1.1996 - accuracy: 0.6085 - val_loss: 0.7786 - val_accuracy: 0.7641
Epoch 6/30
155/155 [==============================] - 157s 1s/step - loss: 1.1668 - accuracy: 0.6254 - val_loss: 0.7942 - val_accuracy: 0.7697
Epoch 7/30
155/155 [==============================] - 157s 1s/step - loss: 1.1511 - accuracy: 0.6345 - val_loss: 0.7742 - val_accuracy: 0.7723